In [5]:
# pip install mlflow

####**Bibliotéques nécessaires**

In [70]:
# Pour la manipulation
import numpy as np
import pandas as pd
# Pour la visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, FunctionTransformer # Encodage des variables catégorielles
from sklearn.impute import SimpleImputer # Gérer les NaNs
from sklearn.model_selection import KFold  # Pour diviser le dataset en pli
from sklearn.metrics import roc_auc_score # évaluation des modèles
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import RocCurveDisplay, classification_report

import lightgbm as lgb

import gc  # Pour libérer la mémoire
import os # Gestion du système de fichiers du drive

# Supprimer les avertissements
import warnings
warnings.filterwarnings('ignore')

import joblib

**train/test_bureau_corrs.csv**
Elle contient les principales informations importantes des clients et leurs prêts.

In [71]:
# Les données d'entrainement
data = pd.read_pickle('train_format.pkl')
print('Dimension des données: ', data.shape)
data.head()

Dimension des données:  (307511, 17)


,SK_ID_CURR,TARGET,EXT_SOURCE_1,EXT_SOURCE_3,EXT_SOURCE_2,DAYS_BIRTH,bureau_DAYS_CREDIT_max,bureau_DAYS_CREDIT_min,bureau_DAYS_CREDIT_UPDATE_mean,bureau_DAYS_CREDIT_mean,bureau_CREDIT_ACTIVE_Closed_mean,client_bureau_balance_STATUS_C_sum_mean,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,bureau_CREDIT_ACTIVE_Active_mean,client_bureau_balance_MONTHS_BALANCE_mean_mean
0,100002,1,0.083008,0.139404,0.262939,-9461.0,-103.0,-1437.0,-500.0,-874.0,0.75,2.875,Cash loans,M,N,0.25,-21.875
1,100003,0,0.311279,NaN,0.622070,-16765.0,-606.0,-2586.0,-816.0,-1401.0,0.75,NaN,Cash loans,F,N,0.25,NaN
2,100004,0,NaN,0.729492,0.556152,-19046.0,-408.0,-1326.0,-532.0,-867.0,1.00,NaN,Revolving loans,M,Y,0.00,NaN
3,100006,0,NaN,NaN,0.650391,-19005.0,NaN,NaN,NaN,NaN,NaN,NaN,Cash loans,F,N,NaN,NaN
4,100007,0,NaN,NaN,0.322754,-19932.0,-1149.0,-1149.0,-783.0,-1149.0,1.00,NaN,Cash loans,M,N,0.00,NaN


Le dataset contiennent **307 511** observations, où chaque observation représente un prêt distinct ayant chacune **39** caractéristiques.
La variable **TARGET** est la cible qu'on cherche à prédire.
  - 0 : le prêt a été remboursé
  - 1 : le prêt n'a pas été remboursé

**Description des variables**

####**Les variables importantes**

In [ ]:
df = data[['SK_ID_CURR', 'TARGET', 'EXT_SOURCE_1', 'EXT_SOURCE_3', 'EXT_SOURCE_2', 'DAYS_BIRTH',
                         'bureau_DAYS_CREDIT_max', 'bureau_DAYS_CREDIT_min', 'bureau_DAYS_CREDIT_UPDATE_mean'
                         , 'bureau_DAYS_CREDIT_mean','bureau_CREDIT_ACTIVE_Closed_mean'
                         , 'client_bureau_balance_STATUS_C_sum_mean', 'NAME_CONTRACT_TYPE'
                         , 'CODE_GENDER', 'FLAG_OWN_CAR', 'bureau_CREDIT_ACTIVE_Active_mean'
                         , 'client_bureau_balance_MONTHS_BALANCE_mean_mean']]

df

,SK_ID_CURR,TARGET,EXT_SOURCE_1,EXT_SOURCE_3,EXT_SOURCE_2,DAYS_BIRTH,bureau_DAYS_CREDIT_max,bureau_DAYS_CREDIT_min,bureau_DAYS_CREDIT_UPDATE_mean,bureau_DAYS_CREDIT_mean,bureau_CREDIT_ACTIVE_Closed_mean,client_bureau_balance_STATUS_C_sum_mean,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,bureau_CREDIT_ACTIVE_Active_mean,client_bureau_balance_MONTHS_BALANCE_mean_mean
0,100002,1,0.083037,0.139376,0.262949,-9461,-103.0,-1437.0,-499.875000,-874.000000,0.750000,2.875000,Cash loans,M,N,0.250000,-21.875000
1,100003,0,0.311267,NaN,0.622246,-16765,-606.0,-2586.0,-816.000000,-1400.750000,0.750000,NaN,Cash loans,F,N,0.250000,NaN
2,100004,0,NaN,0.729567,0.555912,-19046,-408.0,-1326.0,-532.000000,-867.000000,1.000000,NaN,Revolving loans,M,Y,0.000000,NaN
3,100006,0,NaN,NaN,0.650442,-19005,NaN,NaN,NaN,NaN,NaN,NaN,Cash loans,F,N,NaN,NaN
4,100007,0,NaN,NaN,0.322738,-19932,-1149.0,-1149.0,-783.000000,-1149.000000,1.000000,NaN,Cash loans,M,N,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,0.145570,NaN,0.681632,-9327,NaN,NaN,NaN,NaN,NaN,NaN,Cash loans,M,N,NaN,NaN
307507,456252,0,NaN,NaN,0.115992,-20775,NaN,NaN,NaN,NaN,NaN,NaN,Cash loans,F,N,NaN,NaN
307508,456253,0,0.744026,0.218859,0.535722,-14966,-713.0,-919.0,-253.250000,-867.500000,0.500000,14.250000,Cash loans,F,N,0.500000,-14.125000
307509,456254,1,NaN,0.661024,0.514163,-11961,-1104.0,-1104.0,-401.000000,-1104.000000,1.000000,29.000000,Cash loans,F,N,0.000000,-18.000000


**Formatage des variables**

In [72]:
# Sauvegarder les format avec pkl
# df.to_pickle(chemin + '/train_format.pkl')

# Chargement des bons format avec pkl
df = pd.read_pickle('train_red_format.pkl')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 244774 entries, 0 to 307510
Data columns (total 14 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   SK_ID_CURR                        244774 non-null  int32  
 1   TARGET                            244774 non-null  int8   
 2   EXT_SOURCE_3                      244774 non-null  float16
 3   EXT_SOURCE_2                      244774 non-null  float16
 4   DAYS_BIRTH                        244774 non-null  float32
 5   bureau_DAYS_CREDIT_max            244774 non-null  float16
 6   bureau_DAYS_CREDIT_min            244774 non-null  float16
 7   bureau_DAYS_CREDIT_UPDATE_mean    244774 non-null  float16
 8   bureau_DAYS_CREDIT_mean           244774 non-null  float16
 9   bureau_CREDIT_ACTIVE_Closed_mean  244774 non-null  float16
 10  NAME_CONTRACT_TYPE                244774 non-null  object 
 11  CODE_GENDER                       244774 non-null  object

In [73]:
(df.isna().mean()*100).sort_values(ascending=False)

SK_ID_CURR                          0.0
TARGET                              0.0
EXT_SOURCE_3                        0.0
EXT_SOURCE_2                        0.0
DAYS_BIRTH                          0.0
bureau_DAYS_CREDIT_max              0.0
bureau_DAYS_CREDIT_min              0.0
bureau_DAYS_CREDIT_UPDATE_mean      0.0
bureau_DAYS_CREDIT_mean             0.0
bureau_CREDIT_ACTIVE_Closed_mean    0.0
NAME_CONTRACT_TYPE                  0.0
CODE_GENDER                         0.0
FLAG_OWN_CAR                        0.0
bureau_CREDIT_ACTIVE_Active_mean    0.0
dtype: float64

In [39]:
df.describe()

,SK_ID_CURR,TARGET,EXT_SOURCE_3,EXT_SOURCE_2,DAYS_BIRTH,bureau_DAYS_CREDIT_max,bureau_DAYS_CREDIT_min,bureau_DAYS_CREDIT_UPDATE_mean,bureau_DAYS_CREDIT_mean,bureau_CREDIT_ACTIVE_Closed_mean,bureau_CREDIT_ACTIVE_Active_mean
count,244774.000000,244774.000000,244774.000000,244774.000000,244774.000000,244774.0,244774.0,244774.0,244774.0,244774.000000,244774.000000
mean,278159.796396,0.077692,NaN,NaN,-16120.775391,NaN,NaN,NaN,NaN,NaN,NaN
std,102822.698534,0.267687,0.000000,0.000000,4299.810547,NaN,NaN,NaN,NaN,0.000000,0.000000
min,100002.000000,0.000000,0.000527,0.000001,-25201.000000,-2922.0,-2922.0,-41888.0,-2922.0,0.000000,0.000000
25%,188996.750000,0.000000,0.368896,0.400940,-19712.000000,-569.0,-2590.0,-763.0,-1415.0,0.399902,0.199951
50%,278054.500000,0.000000,0.535156,0.569824,-15814.000000,-284.0,-1853.0,-476.0,-1045.0,0.615234,0.375000
75%,367206.750000,0.000000,0.666016,0.665039,-12584.000000,-137.0,-1061.0,-211.5,-666.5,0.799805,0.600098
max,456255.000000,1.000000,0.895996,0.854980,-7676.000000,0.0,-2.0,14.0,-2.0,1.000000,1.000000


In [28]:
df.select_dtypes(include='object').describe()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR
count,244774,244774,244774
unique,2,3,2
top,Cash loans,F,N
freq,221950,162805,159072


**EDA**

In [76]:
def perform_eda(df):
    '''
    On réalise un analyse exploratoire sur le df et on sauve en image.

    entrée :
            df_path : le chemin du df

    sortie :
            None
    '''
    # Transformer les valeurs en valeurs absolues
    df_copy = df.copy()
    # df_copy['DAYS_BIRTH'] = abs(df_copy['DAYS_BIRTH'])
    # Informations sur l'âge dans un DataFrame séparé
    age_data = df_copy[['TARGET', 'DAYS_BIRTH']]

    # Ajouter une colonne contenant l'âge en années
    age_data['YEARS_BIRTH'] = age_data['DAYS_BIRTH'] / -365

    # Diviser les données d'âge par intervalles de 5 ans
    age_data['YEARS_BINNED'] = pd.cut(age_data['YEARS_BIRTH'], bins = np.linspace(20, 70, num = 11))

    # Regrouper les données par tranches d'âge et calculer les moyennes pour chaque tranche d'âge
    age_groups = age_data.groupby('YEARS_BINNED').mean()

   # On réinitialise l'index pour rendre YEARS_BINNED une colonne
    age_groups_reset = age_groups.reset_index()
    # Convertir les tranches d'âge en chaînes pour les utiliser comme catégories
    age_groups_reset['YEARS_BINNED'] = age_groups_reset['YEARS_BINNED'].astype(str)

    # Visualisation
    fig = px.bar(
        age_groups_reset,
        x='YEARS_BINNED',  # Tranches d'âge
        y='TARGET',        # Moyenne du taux de défaut
        text='TARGET',     # Afficher les valeurs sur les barres
        labels={'TARGET': "Défaut de remboursement (%)', 'YEARS_BINNED': 'Tranche d'âge (années)"},
        title="Défaut de remboursement par tranche d'âge"
    )

    # Modifier les valeurs pour afficher le pourcentage
    fig.update_traces(texttemplate='%{text:.2%}', textposition='outside', marker_color='lightskyblue')
    fig.update_yaxes(tickformat=".0%", title_text="Défaut de remboursement (%)")
    fig.update_layout(xaxis_title="Tranche d'âge (années)", title_x=0.5)

    # Affichage
    fig.show();


In [77]:
perform_eda(df)

**ANOVA** ?

#### **Gestion des NANs**

**Spliter les variables**

In [31]:
df_copy = df.copy()
df_copy.shape

(244774, 14)

In [42]:
# On convertie les jours en années
df_copy[['DAYS_BIRTH','bureau_DAYS_CREDIT_max', 'bureau_DAYS_CREDIT_min', 'bureau_DAYS_CREDIT_UPDATE_mean'
     , 'bureau_DAYS_CREDIT_mean']] = df_copy[['DAYS_BIRTH','bureau_DAYS_CREDIT_max', 'bureau_DAYS_CREDIT_min'
                                            , 'bureau_DAYS_CREDIT_UPDATE_mean', 'bureau_DAYS_CREDIT_mean']] /-365

df_copy

,SK_ID_CURR,TARGET,EXT_SOURCE_3,EXT_SOURCE_2,DAYS_BIRTH,bureau_DAYS_CREDIT_max,bureau_DAYS_CREDIT_min,bureau_DAYS_CREDIT_UPDATE_mean,bureau_DAYS_CREDIT_mean,bureau_CREDIT_ACTIVE_Closed_mean,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,bureau_CREDIT_ACTIVE_Active_mean
0,100002,1,0.139404,0.262939,25.920547,0.282192,3.936986,1.369863,2.394521,0.750000,Cash loans,M,N,0.250000
2,100004,0,0.729492,0.556152,52.180820,1.117808,3.632877,1.457534,2.375342,1.000000,Revolving loans,M,Y,0.000000
5,100008,0,0.621094,0.354248,46.413700,0.213699,3.005479,1.673973,2.075342,0.666504,Cash loans,M,N,0.333252
6,100009,0,0.491943,0.724121,37.747944,0.654795,7.895890,2.332877,3.484931,0.777832,Cash loans,F,Y,0.222168
7,100010,0,0.540527,0.714355,51.643837,3.117808,7.506849,1.583562,5.315069,0.500000,Cash loans,M,Y,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307503,456247,0,0.609375,0.501465,32.520550,0.786301,6.800000,2.106849,2.857534,0.727051,Cash loans,F,N,0.272705
307505,456249,0,0.742188,0.346436,66.805481,1.323288,7.430137,2.917808,4.567123,0.846191,Cash loans,F,N,0.153809
307508,456253,0,0.218872,0.535645,41.002739,1.953425,2.517808,0.693836,2.376712,0.500000,Cash loans,F,N,0.500000
307509,456254,1,0.661133,0.514160,32.769863,3.024657,3.024657,1.098630,3.024657,1.000000,Cash loans,F,N,0.000000


In [60]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 244774 entries, 0 to 307510
Data columns (total 14 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   SK_ID_CURR                        244774 non-null  int32  
 1   TARGET                            244774 non-null  int8   
 2   EXT_SOURCE_3                      244774 non-null  float16
 3   EXT_SOURCE_2                      244774 non-null  float16
 4   DAYS_BIRTH                        244774 non-null  float32
 5   bureau_DAYS_CREDIT_max            244774 non-null  float32
 6   bureau_DAYS_CREDIT_min            244774 non-null  float32
 7   bureau_DAYS_CREDIT_UPDATE_mean    244774 non-null  float32
 8   bureau_DAYS_CREDIT_mean           244774 non-null  float32
 9   bureau_CREDIT_ACTIVE_Closed_mean  244774 non-null  float16
 10  NAME_CONTRACT_TYPE                244774 non-null  object 
 11  CODE_GENDER                       244774 non-null  object

In [43]:
quant_cols = df_copy.select_dtypes(exclude='object').columns.to_list()
cat_cols = df_copy.select_dtypes(include='object').columns.to_list()

In [44]:
display(quant_cols)
print(cat_cols)

['SK_ID_CURR',
 'TARGET',
 'EXT_SOURCE_3',
 'EXT_SOURCE_2',
 'DAYS_BIRTH',
 'bureau_DAYS_CREDIT_max',
 'bureau_DAYS_CREDIT_min',
 'bureau_DAYS_CREDIT_UPDATE_mean',
 'bureau_DAYS_CREDIT_mean',
 'bureau_CREDIT_ACTIVE_Closed_mean',
 'bureau_CREDIT_ACTIVE_Active_mean']

['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR']


In [45]:
# La Matrice de corrélation
train_imp_copy = df_copy.copy()
corr = train_imp_copy.drop('SK_ID_CURR',axis = 1).corr(numeric_only=True)
corr

,TARGET,EXT_SOURCE_3,EXT_SOURCE_2,DAYS_BIRTH,bureau_DAYS_CREDIT_max,bureau_DAYS_CREDIT_min,bureau_DAYS_CREDIT_UPDATE_mean,bureau_DAYS_CREDIT_mean,bureau_CREDIT_ACTIVE_Closed_mean,bureau_CREDIT_ACTIVE_Active_mean
TARGET,1.000000,-0.179914,-0.154030,-0.078630,-0.052004,-0.077731,-0.073591,-0.094172,-0.082922,0.081261
EXT_SOURCE_3,-0.179914,1.000000,0.109073,0.204163,0.390192,0.226359,0.334620,0.401751,0.361213,-0.357497
EXT_SOURCE_2,-0.154030,0.109073,1.000000,0.084194,0.054645,0.103482,0.058805,0.111642,0.045414,-0.044454
DAYS_BIRTH,-0.078630,0.204163,0.084194,1.000000,0.089227,0.225819,0.163050,0.207159,0.114713,-0.115142
bureau_DAYS_CREDIT_max,-0.052004,0.390192,0.054645,0.089227,1.000000,0.175028,0.420383,0.626561,0.307585,-0.313374
bureau_DAYS_CREDIT_min,-0.077731,0.226359,0.103482,0.225819,0.175028,1.000000,0.634656,0.809478,0.419884,-0.424992
bureau_DAYS_CREDIT_UPDATE_mean,-0.073591,0.334620,0.058805,0.163050,0.420383,0.634656,1.000000,0.749378,0.558850,-0.563295
bureau_DAYS_CREDIT_mean,-0.094172,0.401751,0.111642,0.207159,0.626561,0.809478,0.749378,1.000000,0.496291,-0.503249
bureau_CREDIT_ACTIVE_Closed_mean,-0.082922,0.361213,0.045414,0.114713,0.307585,0.419884,0.558850,0.496291,1.000000,-0.993635
bureau_CREDIT_ACTIVE_Active_mean,0.081261,-0.357497,-0.044454,-0.115142,-0.313374,-0.424992,-0.563295,-0.503249,-0.993635,1.000000


**Pipeline**

In [35]:
numeric_features = quant_cols
categorical_features = cat_cols

**Modelisation**

**Spliter le dataset**

In [61]:
#  spliter le daataset en trois jeux
train, test = train_test_split(df_copy, test_size= 0.3, random_state=123)

test, validation = train_test_split(test, test_size= 0.5, random_state=123)

In [37]:
print(train.shape)
print(test.shape)
print(validation.shape)

(171341, 14)
(36716, 14)
(36717, 14)


In [46]:
# # Enregistrement des différents jeux de données
# train.to_csv('train.csv', index = False)
# test.to_csv('test.csv', index = False)
# validation.to_csv('validation.csv', index = False)

**Spliter le trainset**

In [62]:
X_train, X_val = train.drop('TARGET', axis = 1), validation.drop('TARGET', axis = 1)
y_train, y_val = train['TARGET'], validation['TARGET']

In [63]:
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

(171341, 13)
(36717, 13)
(171341,)
(36717,)


In [64]:
X_train.head()

,SK_ID_CURR,EXT_SOURCE_3,EXT_SOURCE_2,DAYS_BIRTH,bureau_DAYS_CREDIT_max,bureau_DAYS_CREDIT_min,bureau_DAYS_CREDIT_UPDATE_mean,bureau_DAYS_CREDIT_mean,bureau_CREDIT_ACTIVE_Closed_mean,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,bureau_CREDIT_ACTIVE_Active_mean
27498,131965,0.331299,0.802246,44.432877,0.463014,2.953425,1.034247,1.708219,0.500000,Cash loans,M,Y,0.500000
231837,368531,0.493896,0.447754,36.616440,1.632877,3.589041,0.688356,2.873973,0.399902,Cash loans,F,N,0.600098
2689,103137,0.713867,0.657227,63.849316,0.432877,6.235616,3.019178,4.578082,0.856934,Cash loans,F,N,0.142822
92134,206983,0.216431,0.220459,25.550686,0.539726,0.871233,0.114127,0.650342,1.000000,Cash loans,F,N,0.000000
175076,302878,0.456055,0.564941,34.608219,0.545205,6.964384,1.830137,3.112329,0.833496,Cash loans,M,Y,0.166626


In [65]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 171341 entries, 27498 to 19766
Data columns (total 13 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   SK_ID_CURR                        171341 non-null  int32  
 1   EXT_SOURCE_3                      171341 non-null  float16
 2   EXT_SOURCE_2                      171341 non-null  float16
 3   DAYS_BIRTH                        171341 non-null  float32
 4   bureau_DAYS_CREDIT_max            171341 non-null  float32
 5   bureau_DAYS_CREDIT_min            171341 non-null  float32
 6   bureau_DAYS_CREDIT_UPDATE_mean    171341 non-null  float32
 7   bureau_DAYS_CREDIT_mean           171341 non-null  float32
 8   bureau_CREDIT_ACTIVE_Closed_mean  171341 non-null  float16
 9   NAME_CONTRACT_TYPE                171341 non-null  object 
 10  CODE_GENDER                       171341 non-null  object 
 11  FLAG_OWN_CAR                      171341 non-null  obj

####**Entrainement**

In [66]:
train_cor, test_cor = train_test_split(df_copy, test_size= 0.3, random_state=123)
test_cor = test_cor.drop('TARGET', axis =1)

print(train_cor.shape)
print(test_cor.shape)

(171341, 14)
(73433, 13)


**F3**

In [67]:
def model(features, test_features, encoding='ohe', n_folds=5, save_path='./models'):
    """
    Entraîne et teste un modèle LightGBM avec validation croisée et le sauvegarde.

    Paramètres
    ----------
    features : pd.DataFrame
        DataFrame contenant les features d'entraînement, avec la colonne TARGET.
    test_features : pd.DataFrame
        DataFrame contenant les features de test (sans TARGET).
    encoding : str, par défaut 'ohe'
        Méthode pour encoder les variables catégorielles :
        'ohe' pour One-Hot Encoding, 'le' pour Label Encoding.
    n_folds : int, par défaut 5
        Nombre de plis pour la validation croisée.
    save_path : str, par défaut './models'
        Répertoire où sauvegarder le modèle global.

    Retourne
    --------
    submission : pd.DataFrame
        DataFrame contenant les IDs et les prédictions pour TARGET.
    metrics : pd.DataFrame
        Métriques d'évaluation (AUC) pour chaque pli et globalement.
    """

    # Création du répertoire de sauvegarde si nécessaire
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    # Extraction des identifiants et des labels
    train_ids = features['SK_ID_CURR']
    test_ids = test_features['SK_ID_CURR']
    labels = features['TARGET']

    # Suppression des colonnes non nécessaires
    features = features.drop(columns=['SK_ID_CURR', 'TARGET'])
    test_features = test_features.drop(columns=['SK_ID_CURR'])

    # Étape 1 : Encodage des variables catégorielles
    if encoding == 'ohe':
        features = pd.get_dummies(features)
        test_features = pd.get_dummies(test_features)
        features, test_features = features.align(test_features, join='inner', axis=1)
        cat_indices = 'auto'
    elif encoding == 'le':
        label_encoder = LabelEncoder()
        cat_indices = []
        for i, col in enumerate(features):
            if features[col].dtype == 'object':
                features[col] = label_encoder.fit_transform(features[col].astype(str))
                test_features[col] = label_encoder.transform(test_features[col].astype(str))
                cat_indices.append(i)
    else:
        raise ValueError("L'encodage doit être 'ohe' ou 'le'.")

    print("Taille des données d'entraînement : ", features.shape)
    print('Taille des données de test : ', test_features.shape)

    # Étape 2 : Conversion en tableaux numpy
    features = np.array(features)
    test_features = np.array(test_features)

    # Création de l'objet KFold
    k_fold = KFold(n_splits=n_folds, shuffle=True, random_state=50)

    # Initialisation des variables
    test_predictions = np.zeros(test_features.shape[0])
    out_of_fold = np.zeros(features.shape[0])
    valid_scores = []
    train_scores = []

    # Étape 3 : Validation croisée
    for train_indices, valid_indices in k_fold.split(features):
        train_features, train_labels = features[train_indices], labels.iloc[train_indices]
        valid_features, valid_labels = features[valid_indices], labels.iloc[valid_indices]

        # Modèle LightGBM
        model = lgb.LGBMClassifier(
            n_estimators=10000,
            objective='binary',
            class_weight='balanced',
            learning_rate=0.05,
            reg_alpha=0.1, reg_lambda=0.1,
            subsample=0.8,
            n_jobs=-1,
            random_state=50
        )

        early_stopping = lgb.early_stopping(stopping_rounds=100, verbose=True)
        evaluation = lgb.log_evaluation(period=200)

        model.fit(
            train_features, train_labels,
            eval_metric='auc',
            eval_set=[(valid_features, valid_labels), (train_features, train_labels)],
            eval_names=['valid', 'train'],
            categorical_feature=cat_indices,
            callbacks=[early_stopping, evaluation]
        )

        best_iteration = model.best_iteration_

        # Prédictions sur le test
        test_predictions += model.predict_proba(test_features, num_iteration=best_iteration)[:, 1] / n_folds

        # Prédictions sur les données de validation
        out_of_fold[valid_indices] = model.predict_proba(valid_features, num_iteration=best_iteration)[:, 1]

        # Scores de validation et d'entraînement
        valid_scores.append(model.best_score_['valid']['auc'])
        train_scores.append(model.best_score_['train']['auc'])

        # Libération de la mémoire
        gc.enable()
        del train_features, valid_features
        gc.collect()

    # Étape 4 : Calcul des métriques globales
    valid_auc = roc_auc_score(labels, out_of_fold)
    valid_scores.append(valid_auc)
    train_scores.append(np.mean(train_scores))

    # Sauvegarde du modèle global (sur l'ensemble des données d'entraînement)
    model.fit(
        features, labels,
        eval_metric='auc',
        categorical_feature=cat_indices
    )
    # model_path = os.path.join(save_path, 'model_global.joblib')
    model_path = os.path.abspath(os.path.join(save_path, 'model_global.joblib'))
    with open(model_path, 'wb') as file:
        joblib.dump(model, file)

    print(f"Modèle global sauvegardé dans : {model_path}")

    # Création des sorties
    soumission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': test_predictions})
    fold_names = list(range(n_folds))
    fold_names.append('overall')
    metriques = pd.DataFrame({'fold': fold_names, 'train': train_scores, 'valid': valid_scores})

    return soumission, metriques


**Resultats du modèle**

In [69]:
soumission, metriques = model(train_cor, test_cor, save_path= '/models')

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (2835115991.py, line 1)

In [53]:
soumission

,SK_ID_CURR,TARGET
177957,306232,0.271928
99030,214981,0.307074
193032,323847,0.283735
248250,387219,0.339964
255001,395066,0.837870
...,...,...
170577,297692,0.469365
167132,293749,0.134691
186928,316698,0.729043
141691,264288,0.578959


In [52]:
metriques

,fold,train,valid
0,0,0.771451,0.744832
1,1,0.785399,0.732448
2,2,0.766867,0.736467
3,3,0.779743,0.731606
4,4,0.766738,0.748838
5,overall,0.774040,0.738581


**Sauvegarde du modèle**

In [56]:
loaded_model = joblib.load('model_global.pkl')

In [58]:
# Sauvegarder le modele dans mon environnement colab
joblib.dump(loaded_model,'LightGBM_model.pkl')

['LightGBM_model.pkl']

####**Déploiement**

In [ ]:
!pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 629.7/629.7 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.1 MB/s eta 0:00:00


#####**Chargement du modèle**

In [ ]:
import mlflow
import mlflow.sklearn

# mlflow.start_run()  # Démarrer une expérimentation dans MLflow

# Charger le modèle sauvegardé
with open(chemin +'/mlflow_model/LightGBM_model.pkl', 'rb') as file:
    loaded_model = joblib.load(file)


#####**Inférer une signature**
Afin de garantir que les données d'entrée d'un modèle sont conformes à ce qui est attendu, les modèles MLflow peuvent inclure des metadatas décrivant les entrées et sorties :
- Model Signature - description des entrées et sorties du modèle
- Model Input Example - exemple d'une entrée valide

La signature du modèle permet de renseigner le nom des colonnes et leurs types afin de vérifier si ils sont similaires lors de la prédiction.

In [ ]:
from mlflow.models.signature import infer_signature

**On utilise la fonction infer_signature pour extraire la signature à partir des données d'entrée et de sortie.**

In [ ]:
# Convert to NumPy arrays
X_np = X_train.to_numpy()
y_np = y_train.to_numpy()
signature = infer_signature(X_np, y_np)

#####**Enregistrer le modèle dans MLflow**

In [ ]:
# Enregistrer le modèle dans MLflow
mlflow.sklearn.log_model(
    sk_model=loaded_model,
    artifact_path="lightgbm_model",  # Chemin dans MLflow
    registered_model_name="LightGBM_Classifier"  # Nom du modèle dans MLflow
)


2024/12/19 23:30:40 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'LightGBM_Classifier'.
Created version '1' of model 'LightGBM_Classifier'.


In [ ]:
# # Terminer l'expérimentation
# mlflow.end_run()

**Python anywhere ou Rander** pour tourner l'API